In [4]:
import random
import numpy as np
from enum import Enum
from sklearn.ensemble import RandomForestRegressor


NOTES = ["C", "C#", "D", "D#", "E", "F", "F#", "G", "G#", "A", "A#", "B"]


class Scales(Enum):
    IONIAN = 0
    DORIAN = 2
    PHRYGIAN = 4
    LYDIAN = 5
    MIXOLYDIAN = 7
    AEOLIAN = 9
    LOCRIAN = 11


PATTERNS_C_IONIAN = [
    [[0, 1, 3], [0, 2, 3], [0, 2, 3], [0, 2, 4], [0, 1, 3], [0, 1, 3]],
    [[0, 2, 4], [1, 2, 4], [1, 2, 4], [1, 3, 4], [0, 2, 4], [0, 2, 4]],
    [[1, 3], [0, 1, 3], [0, 1, 3], [0, 2, 3], [1, 3, 4], [1, 3]],
    [[0, 2, 4], [0, 2, 4], [0, 2, 4], [1, 2, 4], [0, 2, 3], [0, 2, 4]],
    [[1, 3, 4], [1, 3, 4], [1, 3], [0, 1, 3], [2, 3, 4], [1, 3, 4]],
    [[0, 2, 3], [0, 2, 3], [0, 2, 4], [0, 2, 4], [0, 1, 3], [0, 2, 3]],
    [[1, 2, 4], [1, 2, 4], [1, 3, 4], [1, 3, 4], [0, 2, 4], [1, 2, 4]],
    [[0, 1, 3], [0, 1, 3], [0, 2, 3], [0, 2, 3], [1, 3], [0, 1, 3]],
    [[0, 2, 4], [0, 2, 4], [1, 2, 4], [1, 2, 4], [0, 2, 4], [0, 2, 4]],
    [[1, 3, 4], [1, 3], [0, 1, 3], [0, 1, 3], [1, 3, 4], [1, 3, 4]],
    [[0, 2, 3], [0, 2, 4], [0, 2, 4], [0, 2, 4], [0, 2, 3], [0, 2, 3]],
    [[1, 2, 4], [1, 3, 4], [1, 3, 4], [1, 3], [1, 2, 4], [1, 2, 4]]]

In [5]:
def get_pattern(key, scale=Scales.IONIAN):
    to_index = (len(NOTES) - NOTES.index(key)) + scale.value
    if to_index >= len(PATTERNS_C_IONIAN):
        to_index = to_index - len(PATTERNS_C_IONIAN)

    return PATTERNS_C_IONIAN[to_index:] + PATTERNS_C_IONIAN[:to_index]

In [6]:
def generate_chord(pattern=None, key="C", scale=Scales.IONIAN, position=0, include_position=False, note_count=3):
    # Generate a random chord
    if pattern is None:
        pattern = get_pattern(key, scale=scale)
    chord = []
    chosen_strings = random.sample(range(6), note_count)
    for i in range(6):
        if i in chosen_strings:
            chord.append(random.choice(pattern[position][i]) + include_position * position)
        else:
            chord.append(-1)
    return chord

In [7]:
def generate_note(pattern=None, key="C", scale=Scales.IONIAN, position=0, include_position=False):
    generate_chord(pattern=pattern, key=key, scale=scale, position=position, include_position=include_position,
                   note_count=1)

In [16]:
get_pattern("C", scale=Scales.DORIAN)

[[[1, 3], [0, 1, 3], [0, 1, 3], [0, 2, 3], [1, 3, 4], [1, 3]],
 [[0, 2, 4], [0, 2, 4], [0, 2, 4], [1, 2, 4], [0, 2, 3], [0, 2, 4]],
 [[1, 3, 4], [1, 3, 4], [1, 3], [0, 1, 3], [2, 3, 4], [1, 3, 4]],
 [[0, 2, 3], [0, 2, 3], [0, 2, 4], [0, 2, 4], [0, 1, 3], [0, 2, 3]],
 [[1, 2, 4], [1, 2, 4], [1, 3, 4], [1, 3, 4], [0, 2, 4], [1, 2, 4]],
 [[0, 1, 3], [0, 1, 3], [0, 2, 3], [0, 2, 3], [1, 3], [0, 1, 3]],
 [[0, 2, 4], [0, 2, 4], [1, 2, 4], [1, 2, 4], [0, 2, 4], [0, 2, 4]],
 [[1, 3, 4], [1, 3], [0, 1, 3], [0, 1, 3], [1, 3, 4], [1, 3, 4]],
 [[0, 2, 3], [0, 2, 4], [0, 2, 4], [0, 2, 4], [0, 2, 3], [0, 2, 3]],
 [[1, 2, 4], [1, 3, 4], [1, 3, 4], [1, 3], [1, 2, 4], [1, 2, 4]],
 [[0, 1, 3], [0, 2, 3], [0, 2, 3], [0, 2, 4], [0, 1, 3], [0, 1, 3]],
 [[0, 2, 4], [1, 2, 4], [1, 2, 4], [1, 3, 4], [0, 2, 4], [0, 2, 4]]]

In [23]:
generate_chord(
    key="D",
    scale=Scales.MIXOLYDIAN,
    position=random.randrange(0, 5),
    include_position=True,
    note_count=random.randrange(2, 5))

[5, -1, 7, 4, -1, -1]

In [3]:
POPULATION = 100
NOTES_PER = 100


def post_process_duration(y_pred):
    # Round duration to the nearest multiple of 0.125
    y_pred[:] = np.round(y_pred[:] / 0.125) * 0.125

    return y_pred


def post_process_velocity(y_pred):
    # Clip velocity to the range 1-127
    y_pred[:] = np.clip(np.round(y_pred[:]), 1, 127)

    return y_pred


def preprocess_input(x):
    # Convert chords to binary
    x = np.array([[''.join([bin(x)[2:].zfill(6) for x in row]) for row in level] for level in (x + 1)])

    return x


# toy dataset
# chordsX = [[chord, position]...]
# chordsY = [[duration, velocity]...]
rand_chords_x = np.array([[generate_chord(position=i % 12, note_count=(random.randrange(1, 5)), include_position=True) for i in range(NOTES_PER)] for _ in range(POPULATION)])
print(rand_chords_x)

rand_chords_x_flat = preprocess_input(rand_chords_x)
print(rand_chords_x_flat)

rand_durations = np.array(
    [[round(random.random() / 0.125) * 0.125 for _ in range(NOTES_PER)] for _ in range(POPULATION)])
rand_velocities = np.array([[random.randrange(50, 127) for _ in range(NOTES_PER)] for _ in range(POPULATION)])
# chordsYFlat = np.reshape(chordsY, (12, 20))
print(rand_durations)
print(rand_velocities)

durRegr = RandomForestRegressor(n_estimators=100)
durRegr.fit(rand_chords_x_flat, rand_durations)
velRegr = RandomForestRegressor(n_estimators=100)
velRegr.fit(rand_chords_x_flat, rand_velocities)

X_test = np.array([[generate_chord(position=i % 12, note_count=(random.randrange(1, 5)), include_position=True) for i in
                    range(NOTES_PER)] for _ in range(POPULATION)])
print(preprocess_input(X_test))
print(post_process_duration(durRegr.predict(preprocess_input(X_test))))
print(post_process_velocity(velRegr.predict(preprocess_input(X_test))))

[[[-1  3 -1  2 -1  0]
  [ 5  2 -1 -1 -1  5]
  [-1  5 -1  2  5  3]
  ...
  [ 5 -1  3  4 -1  5]
  [ 5  2 -1  2 -1  3]
  [-1 -1  7 -1  6 -1]]

 [[ 0  2 -1 -1 -1  0]
  [-1  3 -1 -1  1 -1]
  [ 3  3 -1 -1  3  5]
  ...
  [ 5 -1  3  5 -1  1]
  [-1  3  3 -1 -1  5]
  [ 5  5  7 -1 -1  5]]

 [[ 1 -1 -1 -1 -1  1]
  [-1 -1 -1 -1  3 -1]
  [ 5 -1 -1  2  6 -1]
  ...
  [ 5  3 -1 -1  5  1]
  [-1 -1 -1  2 -1  3]
  [-1 -1  5  7  5 -1]]

 ...

 [[ 1 -1  3  0  0 -1]
  [ 1 -1  5  5 -1  5]
  [-1  5 -1 -1 -1 -1]
  ...
  [-1  2  5 -1 -1 -1]
  [-1  2  3 -1  6  3]
  [-1 -1 -1 -1 -1  3]]

 [[ 1 -1 -1 -1 -1  3]
  [ 1 -1 -1 -1 -1 -1]
  [-1 -1 -1 -1  5 -1]
  ...
  [-1 -1  3  4  3 -1]
  [ 3  5  5 -1 -1  3]
  [ 7 -1  7 -1  3 -1]]

 [[ 3 -1  0 -1 -1  1]
  [ 1  5 -1 -1 -1 -1]
  [ 5 -1 -1 -1 -1  5]
  ...
  [ 3  2 -1 -1  5 -1]
  [-1  5  5 -1  6 -1]
  [-1  7 -1  5 -1  5]]]
[['000000000100000000000011000000000001'
  '000110000011000000000000000000000110'
  '000000000110000000000011000110000100' ...
  '000110000000000100000101

'\n\nTODO: IDEAS\n\n- Train an ML algorithm on a dataset of chords arrays (like above) and their corresponding duration and velocity\n\n'